In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Loading main data

In [2]:
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [4]:
data.shape

(1162, 6)

In [5]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,52.651001,53.799000,52.651001,53.660500,53.660500,31766000
2018-01-03,53.696499,54.805000,53.671501,54.576000,54.576000,31318000
2018-01-04,54.854500,55.203999,54.713001,54.787998,54.787998,26052000


In [6]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [7]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [8]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-08-10,118.779999,120.919998,118.410004,119.699997,119.699997,30930400
2022-08-11,121.279999,121.489998,118.559998,118.839996,118.839996,24293700
2022-08-12,120.260002,121.680000,119.459999,121.680000,121.680000,19074200


In [9]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [10]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000,1.162000e+03
mean,84.535412,85.456098,83.607253,84.558182,84.558182,3.573023e+07
std,31.667467,31.963695,31.302395,31.629023,31.629023,1.628298e+07
min,49.216000,50.605999,48.882999,49.233501,49.233501,9.312000e+06
25%,58.051251,58.848626,57.596624,58.358748,58.358748,2.527550e+07
50%,71.201500,71.846748,70.134003,71.003498,71.003498,3.169800e+07
75%,113.366123,115.280624,112.543877,113.942625,113.942625,4.093300e+07
max,151.250000,151.546494,148.899002,149.838501,149.838501,1.331780e+08


Understanding Trends with in the Data

In [11]:
import plotly.graph_objects as go

In [12]:
# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [13]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [14]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [15]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,53.660500,31766000
2018-01-03,54.576000,31318000
2018-01-04,54.787998,26052000


In [16]:
data.index = pd.to_datetime(data.index)

Scrapping extra information

In [17]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=STOCH&symbol=GOOGL&interval=daily&apikey=4QJHZ8EE7RTFMD5C')
response = response.json()
stoch_data = pd.DataFrame.from_dict(response['Technical Analysis: STOCH'] , orient='index')
stoch_data = stoch_data[stoch_data.index >= '2018-01-01']
stoch_data['SlowD'] = stoch_data['SlowD'].astype(np.float64)
stoch_data['SlowK'] = stoch_data['SlowK'].astype(np.float64)
stoch_data.index = pd.to_datetime(stoch_data.index)
data = data.merge(stoch_data, left_index=True, right_index=True, how='left')

In [18]:
response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=10&series_type=close&apikey=4QJHZ8EE7RTFMD5C') 
response = response.json()
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)
rsi_data.index = pd.to_datetime(rsi_data.index)
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [19]:
response = requests.get('https://www.alphavantage.co/query?function=EMA&symbol=GOOGL&interval=daily&time_period=10&series_type=close&apikey=4QJHZ8EE7RTFMD5C') 
response = response.json()
ema_data = pd.DataFrame.from_dict(response['Technical Analysis: EMA'] , orient='index')
ema_data = ema_data[ema_data.index >= '2018-01-01']
ema_data['EMA'] = ema_data['EMA'].astype(np.float64)
ema_data.index = pd.to_datetime(ema_data.index)
data = data.merge(ema_data, left_index=True, right_index=True, how='inner')

In [20]:
data.head()

,Close,Volume,SlowK,SlowD,RSI,EMA
Date,,,,,,
2018-01-02,53.660500,31766000,34.1426,18.9682,60.4790,53.1742
2018-01-03,54.576000,31318000,60.4096,33.9942,69.1043,53.4291
2018-01-04,54.787998,26052000,87.1184,60.5569,70.7470,53.6761
2018-01-05,55.514500,30250000,89.2833,78.9371,75.6721,54.0104
2018-01-08,55.710499,24644000,90.3062,88.9027,76.8409,54.3195


In [21]:
data.isnull().sum()

Close     0
Volume    0
SlowK     0
SlowD     0
RSI       0
EMA       0
dtype: int64

In [22]:
data['Stoch%'] = (data['SlowK']/data['SlowD']) - 1

In [23]:
data.head()

,Close,Volume,SlowK,SlowD,RSI,EMA,Stoch%
Date,,,,,,,
2018-01-02,53.660500,31766000,34.1426,18.9682,60.4790,53.1742,0.799992
2018-01-03,54.576000,31318000,60.4096,33.9942,69.1043,53.4291,0.777056
2018-01-04,54.787998,26052000,87.1184,60.5569,70.7470,53.6761,0.438621
2018-01-05,55.514500,30250000,89.2833,78.9371,75.6721,54.0104,0.131069
2018-01-08,55.710499,24644000,90.3062,88.9027,76.8409,54.3195,0.015787


In [24]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2022-01-01')].shape[0]

In [25]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [26]:
X , Y = CreateFeatures_and_Targets(data , 14)

  0%|          | 0/1148 [00:00<?, ?it/s]

In [27]:
# Check the shapes
X.shape , Y.shape

((1148, 14, 7), (1148,))

In [28]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [29]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((994, 14, 7), (994,))

In [30]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((154, 14, 7), (154,))

In [31]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [32]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [33]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [34]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [35]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [36]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [38]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [39]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
994/994 [==============================] - 79s 71ms/step - loss: 0.0060 - val_loss: 0.0300 - lr: 0.0020
Epoch 2/10
994/994 [==============================] - 65s 65ms/step - loss: 0.0081 - val_loss: 0.0450 - lr: 0.0020
Epoch 3/10
994/994 [==============================] - 64s 64ms/step - loss: 0.0071 - val_loss: 0.0258 - lr: 0.0020
Epoch 4/10
994/994 [==============================] - 64s 65ms/step - loss: 0.0051 - val_loss: 0.0087 - lr: 0.0020
Epoch 5/10
994/994 [==============================] - 66s 66ms/step - loss: 0.0042 - val_loss: 0.0058 - lr: 0.0020
Epoch 6/10
994/994 [==============================] - 64s 65ms/step - loss: 0.0037 - val_loss: 0.0065 - lr: 0.0020
Epoch 7/10
994/994 [==============================] - 64s 65ms/step - loss: 0.0037 - val_loss: 0.0110 - lr: 0.0020
Epoch 8/10
994/994 [==============================] - 64s 64ms/step - loss: 0.0033 - val_loss: 0.0029 - lr: 0.0020
Epoch 9/10
994/994 [==============================] - 64s 65ms/step - loss: 0.00

In [40]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [41]:
Predictions = model.predict(Xtest)

In [42]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [43]:
Predictions.shape

(154, 1)

In [44]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [45]:
# Check the Predictions vs Actual
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [46]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [47]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [48]:
Predictions = model.predict(Total_features)

In [49]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [50]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [51]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [52]:
# Save and Load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

Realtime Prediction

In [55]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 14):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [59]:
PredictStockPrice(loaded_model , data , '2022-08-12')

120.65962

In [ ]:
stock_price = data['Close']

def PredictStockPrice(Model , DataFrame , ForecastDate , feature_length = 20):
    for i in range(datetime.strptime(ForecastDate, '%y/%m/%d') - DataFrame.index[-1]):
      Features = DataFrame.iloc[-14 : -1,:].values
      Features = np.expand_dims(Features , axis = 0)
      Features = Feature_Scaler.transform(Features)
      Prediction = Model.predict(Features)
      Prediction = Target_Scaler.inverse_transform(Prediction)
      df_forecast = pd.DataFrame(Prediction, index=DataFrame.index[-1]+i+1)
    return Prediction[0][0]

In [60]:
data.index[-1]

Timestamp('2022-08-12 00:00:00')

In [62]:
for i in range (5):
  print(i)

0
1
2
3
4
